# Yet another example of Agency.

In [1]:
import os
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=UserWarning)

In [2]:
import os
import textwrap
import uuid

import shapely
from gait import Agency, Agent, ObserverLoguru, OperatorLiteLLM, Scratchpad
from rich.pretty import pprint

In [3]:
suffix = "gpt-4.1-mini"
model = f"azure/{suffix}"
api_base = f"{os.environ['AZURE_API_URL']}/{suffix}"

## Define the functions needed by the agents.

In [4]:
def buffer_geometry(
    geom_uuid: str,
    distance_in_meters: float,
    scratchpad: Scratchpad,
) -> str:
    """Buffer a geometry referenced by a UUID by a distance in meters.
    Return the buffered geometry UUID.

    :param geom_uuid: UUID of a geometry to buffer.
    :param distance_in_meters: The buffer distance in meters.
    :param scratchpad: Scratchpad instance injected at runtime.
    """
    geom = scratchpad[geom_uuid]
    buff = shapely.buffer(geom, distance_in_meters)
    buff_uuid = str(uuid.uuid4())
    scratchpad[buff_uuid] = buff
    return buff_uuid

In [5]:
buffer_agent = Agent(
    model=model,
    name="BufferAgent",
    description="Buffer a geometry UUID by a distance",
    instructions="Buffer a geometry referenced by a UUID by a distance in meters.",
    functions=[buffer_geometry],
    api_base=api_base,
    temperature=0.0,
    tool_choice="required",
)

In [6]:
def wkt_to_geometry(
    wkt: str,
    scratchpad: Scratchpad,
) -> str:
    """Parse WKT string into a geometry UUID.
    The following are WKT string samples:
    POINT (30 10)
    LINESTRING (30 10, 10 30, 40 40)
    POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))

    :param wkt: The well known text to convert to a geometry UUID.
    :param scratchpad:  Scratchpad instance injected at runtime.
    """
    geom = shapely.from_wkt(wkt, on_invalid="ignore")
    geom_uuid = str(uuid.uuid4())
    scratchpad[geom_uuid] = geom
    return geom_uuid

In [7]:
wkt_agent = Agent(
    model=model,
    name="WKTAgent",
    description=textwrap.dedent(
        """
    Convert WKT string to geometry UUID.
    
    The following are WKT string samples:
    POINT (30 10)
    LINESTRING (30 10, 10 30, 40 40)
    POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))    
    """
    ),
    functions=[wkt_to_geometry],
    api_base=api_base,
    temperature=0.0,
    tool_choice="required",
)

In [8]:
report_agent = Agent(
    model=model,
    name="ReportAgent",
    description="""Use this agent to report the final answer. Make sure to suffix the final answer with <ANSWER/>""",
    api_base=api_base,
    temperature=0.2,
    stop=["<ANSWER/>"],
)

## Define the operator with reference to all the handoff agents.

In [9]:
operator = OperatorLiteLLM(
    model,
    agents=[report_agent, buffer_agent, wkt_agent],
    api_base=api_base,
    tool_choice="required",
)

## Run the agency.

In [10]:
agency = Agency(operator, observer=ObserverLoguru())

for _ in agency("Get the UUID of the buffer of WKT POINT(10 5) by 10 meters."):
    if _.agent == report_agent:
        agency.terminate()
        print(_.content)

12:49:35 | INFO | GAIT started.
12:49:36 | INFO | Iteration 1 (WKTAgent) started.
12:49:37 | INFO | Content: None
12:49:37 | INFO | Function: wkt_to_geometry({"wkt":"POINT(10 5)"})
12:49:37 | INFO | Observation: d21ef603-e1ce-4681-b2b0-7f19d5e9266e
12:49:37 | INFO | Iteration 2 (BufferAgent) started.
12:49:37 | INFO | Handoff from WKTAgent to BufferAgent.
12:49:38 | INFO | Content: None
12:49:38 | INFO | Function: buffer_geometry({"geom_uuid":"d21ef603-e1ce-4681-b2b0-7f19d5e9266e","distance_in_meters":10})
12:49:38 | INFO | Observation: 03f3d348-1194-45ab-bb29-ef412d5ea7e3
12:49:38 | INFO | Iteration 3 (ReportAgent) started.
12:49:38 | INFO | Handoff from BufferAgent to ReportAgent.
12:49:39 | INFO | Content: The UUID of the buffer of WKT POINT(10 5) by 10 meters is 03f3d348-1194-45ab-bb29-ef412d5ea7e3
12:49:39 | INFO | GAIT ended after 3 iterations.


The UUID of the buffer of WKT POINT(10 5) by 10 meters is 03f3d348-1194-45ab-bb29-ef412d5ea7e3


### Get the messages in the dialog.

In [11]:
for _ in agency.dialog:
    pprint(_, expand_all=True)

{
│   'role': 'user',
│   'content': 'Get the UUID of the buffer of WKT POINT(10 5) by 10 meters.'
}

Message(
│   content=None,
│   role='assistant',
│   tool_calls=[
│   │   {
│   │   │   'function': {
│   │   │   │   'arguments': '{"wkt":"POINT(10 5)"}',
│   │   │   │   'name': 'wkt_to_geometry'
│   │   │   },
│   │   │   'id': 'call_cKg4ZW0uDyXyHD3ydODNhiHV',
│   │   │   'type': 'function'
│   │   }
│   ],
│   function_call=None,
│   provider_specific_fields={
│   │   'refusal': None
│   },
│   annotations=[]
)

{
│   'role': 'tool',
│   'content': 'd21ef603-e1ce-4681-b2b0-7f19d5e9266e',
│   'name': 'wkt_to_geometry',
│   'tool_call_id': 'call_cKg4ZW0uDyXyHD3ydODNhiHV'
}

Message(
│   content=None,
│   role='assistant',
│   tool_calls=[
│   │   {
│   │   │   'function': {
│   │   │   │   'arguments': '{"geom_uuid":"d21ef603-e1ce-4681-b2b0-7f19d5e9266e","distance_in_meters":10}',
│   │   │   │   'name': 'buffer_geometry'
│   │   │   },
│   │   │   'id': 'call_a0jqaOLZKf04h3W0oNltXkaW',
│   │   │   'type': 'function'
│   │   }
│   ],
│   function_call=None,
│   provider_specific_fields={
│   │   'refusal': None
│   },
│   annotations=[]
)

{
│   'role': 'tool',
│   'content': '03f3d348-1194-45ab-bb29-ef412d5ea7e3',
│   'name': 'buffer_geometry',
│   'tool_call_id': 'call_a0jqaOLZKf04h3W0oNltXkaW'
}

Message(
│   content='The UUID of the buffer of WKT POINT(10 5) by 10 meters is 03f3d348-1194-45ab-bb29-ef412d5ea7e3',
│   role='assistant',
│   tool_calls=None,
│   function_call=None,
│   provider_specific_fields={
│   │   'refusal': None
│   },
│   annotations=[]
)

### Get the scratchpad content.

In [12]:
for _ in agency.scratchpad:
    pprint(_, expand_all=True)

(
│   'd21ef603-e1ce-4681-b2b0-7f19d5e9266e',
│   <POINT (10 5)>
)

(
│   '03f3d348-1194-45ab-bb29-ef412d5ea7e3',
│   <POLYGON ((20 5, 19.808 3.049, 19.239 1.173, 18.315 -0.556, 17.071 -2.071, 1...>
)